In [2]:
# model = Equation (coeff, intercept)

In [ ]:
# save model
# portable ship
# load model
# predict

In [ ]:
train 
test

Hackathon
train+test -> Preprocessing (Hackathon)

train - flight price - carrier - indigo, airindia, jet

testflight price - carrier - indigo, airindia, jet, vistara


combined and onehot -> no problem in forseen

missing value -> imputation -> mean -> train+test


Realworld
Train - Preprocessing



Test - We don't have - Preprocess -> predict


Extra Burden on the devepoler to repeat the step of preprocessing 

Ship the processing code with the model



train - flight price - carrier - indigo, airindia, jet

test - flight price - carrier - indigo, airindia, jet, vistara


separatly did the onehot -> problem 


missing value -> imputation -> mean -> train



In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression

import joblib

In [2]:
data = pd.DataFrame(
{
    'name':['Maruti','honda','maruti','toyOta','hondA','toyota'],
    'fuel_type':['petrol','diesel','petrol','diesel','petrol','diesel'],
    'km_driven' :[2000,4000,100,6344,1234,np.NaN],
    'price' :[200000,400000,424232,800000,500000,900000]
    
}
)
 

In [3]:
data

,name,fuel_type,km_driven,price
0,Maruti,petrol,2000.0,200000
1,honda,diesel,4000.0,400000
2,maruti,petrol,100.0,424232
3,toyOta,diesel,6344.0,800000
4,hondA,petrol,1234.0,500000
5,toyota,diesel,NaN,900000


In [4]:
data['name'].unique()

array(['Maruti', 'honda', 'maruti', 'toyOta', 'hondA', 'toyota'],
      dtype=object)

In [31]:
class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        #print('\n>>>>>>>init() called.\n')
        pass
    def fit(self, X, y = None):
        #print('\n>>>>>>>fit() called.\n')
        return self
    def transform(self, X, y = None):
        #print('\n>>>>>>>transform() called.\n')
        #print("\n>>>> Input : ",X)
        X_ = X.applymap(lambda x: x.lower())
        #print("\n>>>> Output : ",X_)
        #print("\n>>>>>>> Custom Transformer Called")
        return X_


In [32]:

preprocess_categorical_encoding = Pipeline([('pre_preprocess', CustomTransformer()),
          ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
         ])

numerical_encoding_scaling = Pipeline([
                                       ('imputation', SimpleImputer(strategy='mean')),
                                       ('scale', MinMaxScaler())])


In [33]:
preprocess_full = ColumnTransformer(
    transformers=
    [
    ('numerical_prprocessing',numerical_encoding_scaling,['km_driven'],),
    ('categoral_preprocessing',preprocess_categorical_encoding,['name','fuel_type'])
    
    ]
)

In [34]:
pipe = Pipeline([
         ('preprocess',preprocess_full),
         ('model',LinearRegression())])

In [35]:
data.drop(columns = 'price')

,name,fuel_type,km_driven
0,Maruti,petrol,2000.0
1,honda,diesel,4000.0
2,maruti,petrol,100.0
3,toyOta,diesel,6344.0
4,hondA,petrol,1234.0
5,toyota,diesel,NaN


In [36]:
y = data['price']

In [37]:
pipe.fit(data.drop(columns = 'price'),y)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('numerical_prprocessing',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['km_driven']),
                                                 ('categoral_preprocessing',
                                                  Pipeline(steps=[('pre_preprocess',
                                                                   CustomTransformer()),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['name', 'fuel_type'])])),
                ('model', LinearRegression())])

In [38]:
from sklearn import set_config
set_config(display='diagram')  
display(pipe)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('numerical_prprocessing',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['km_driven']),
                                                 ('categoral_preprocessing',
                                                  Pipeline(steps=[('pre_preprocess',
                                                                   CustomTransformer()),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['name', 'fuel_type'])])),
                ('model', LinearRegression())])

In [39]:
new_test = ['Toyota','Petrol',np.NaN]

In [40]:
new_ = pd.DataFrame(new_test).T

In [41]:
new_.columns = ['name','fuel_type', 'km_driven']

In [42]:
new_

,name,fuel_type,km_driven
0,Toyota,Petrol,NaN


In [43]:
pipe.predict(new_)

array([904492.0686354])

In [44]:
joblib.dump(pipe,'preprocess_model.pkl')

['preprocess_model.pkl']

In [45]:
pipe

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('numerical_prprocessing',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['km_driven']),
                                                 ('categoral_preprocessing',
                                                  Pipeline(steps=[('pre_preprocess',
                                                                   CustomTransformer()),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['name', 'fuel_type'])])),
                ('model', LinearRegression())])

In [ ]:
# Cross validation

In [46]:
from sklearn.model_selection import cross_val_score

In [55]:
cross_val_score(estimator = pipe,X = data.drop(columns = 'price'),y = y, cv = 3, 
                scoring = 'r2')

array([-5.18713474,  0.32906229, -0.57434347])

In [62]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [59]:
pipe_grid = Pipeline([
         ('preprocess',preprocess_full),
         ('model',Lasso())])

In [60]:
pipe_grid

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('numerical_prprocessing',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['km_driven']),
                                                 ('categoral_preprocessing',
                                                  Pipeline(steps=[('pre_preprocess',
                                                                   CustomTransformer()),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['name', 'fuel_type'])])),
                ('model', Lasso())])

In [95]:
params = {'model__alpha':[1,2,3,4,5,6,7,8,9,10,20,30],
          'model__normalize':[True, False],
          'model__fit_intercept' : [True, False]
         }


In [106]:
grid = GridSearchCV(estimator = pipe_grid, param_grid = params,scoring = 'r2', cv = 3, 
                    verbose= 4)


In [107]:
grid.fit(data.drop(columns = 'price'),y)

Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV 1/3] END model__alpha=1, model__fit_intercept=True, model__normalize=True;, score=-0.551 total time=   0.0s
[CV 2/3] END model__alpha=1, model__fit_intercept=True, model__normalize=True;, score=0.400 total time=   0.0s
[CV 3/3] END model__alpha=1, model__fit_intercept=True, model__normalize=True;, score=-3.149 total time=   0.0s
[CV 1/3] END model__alpha=1, model__fit_intercept=True, model__normalize=False;, score=-0.551 total time=   0.0s
[CV 2/3] END model__alpha=1, model__fit_intercept=True, model__normalize=False;, score=0.400 total time=   0.0s
[CV 3/3] END model__alpha=1, model__fit_intercept=True, model__normalize=False;, score=-3.153 total time=   0.0s
[CV 1/3] END model__alpha=1, model__fit_intercept=False, model__normalize=True;, score=-0.553 total time=   0.0s
[CV 2/3] END model__alpha=1, model__fit_intercept=False, model__normalize=True;, score=-11.712 total time=   0.0s
[CV 3/3] END model__alpha=1, model__fi

/Users/mohamednoordeenalaudeen/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/Users/mohamednoordeenalaudeen/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a

[CV 1/3] END model__alpha=3, model__fit_intercept=False, model__normalize=True;, score=-0.553 total time=   0.0s
[CV 2/3] END model__alpha=3, model__fit_intercept=False, model__normalize=True;, score=-11.707 total time=   0.0s
[CV 3/3] END model__alpha=3, model__fit_intercept=False, model__normalize=True;, score=-0.594 total time=   0.0s
[CV 1/3] END model__alpha=3, model__fit_intercept=False, model__normalize=False;, score=-0.553 total time=   0.0s
[CV 2/3] END model__alpha=3, model__fit_intercept=False, model__normalize=False;, score=-11.707 total time=   0.0s
[CV 3/3] END model__alpha=3, model__fit_intercept=False, model__normalize=False;, score=-0.594 total time=   0.0s
[CV 1/3] END model__alpha=4, model__fit_intercept=True, model__normalize=True;, score=-0.552 total time=   0.0s
[CV 2/3] END model__alpha=4, model__fit_intercept=True, model__normalize=True;, score=0.400 total time=   0.0s
[CV 3/3] END model__alpha=4, model__fit_intercept=True, model__normalize=True;, score=-2.782 t

/Users/mohamednoordeenalaudeen/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/Users/mohamednoordeenalaudeen/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence t

[CV 3/3] END model__alpha=6, model__fit_intercept=True, model__normalize=True;, score=-2.596 total time=   0.0s
[CV 1/3] END model__alpha=6, model__fit_intercept=True, model__normalize=False;, score=-0.553 total time=   0.0s
[CV 2/3] END model__alpha=6, model__fit_intercept=True, model__normalize=False;, score=0.400 total time=   0.0s
[CV 3/3] END model__alpha=6, model__fit_intercept=True, model__normalize=False;, score=-2.594 total time=   0.0s
[CV 1/3] END model__alpha=6, model__fit_intercept=False, model__normalize=True;, score=-0.553 total time=   0.0s
[CV 2/3] END model__alpha=6, model__fit_intercept=False, model__normalize=True;, score=-11.701 total time=   0.0s
[CV 3/3] END model__alpha=6, model__fit_intercept=False, model__normalize=True;, score=-0.585 total time=   0.0s
[CV 1/3] END model__alpha=6, model__fit_intercept=False, model__normalize=False;, score=-0.553 total time=   0.0s
[CV 2/3] END model__alpha=6, model__fit_intercept=False, model__normalize=False;, score=-11.701 

...

[CV 2/3] END model__alpha=8, model__fit_intercept=False, model__normalize=False;, score=-11.694 total time=   0.0s
[CV 3/3] END model__alpha=8, model__fit_intercept=False, model__normalize=False;, score=-0.579 total time=   0.0s
[CV 1/3] END model__alpha=9, model__fit_intercept=True, model__normalize=True;, score=-0.553 total time=   0.0s
[CV 2/3] END model__alpha=9, model__fit_intercept=True, model__normalize=True;, score=0.568 total time=   0.0s
[CV 3/3] END model__alpha=9, model__fit_intercept=True, model__normalize=True;, score=-2.329 total time=   0.0s
[CV 1/3] END model__alpha=9, model__fit_intercept=True, model__normalize=False;, score=-0.553 total time=   0.0s
[CV 2/3] END model__alpha=9, model__fit_intercept=True, model__normalize=False;, score=0.401 total time=   0.0s
[CV 3/3] END model__alpha=9, model__fit_intercept=True, model__normalize=False;, score=-2.326 total time=   0.0s
[CV 1/3] END model__alpha=9, model__fit_intercept=False, model__normalize=True;, score=-0.553 tota

/Users/mohamednoordeenalaudeen/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/Users/mohamednoordeenalaudeen/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a

[CV 1/3] END model__alpha=20, model__fit_intercept=False, model__normalize=True;, score=-0.553 total time=   0.0s
[CV 2/3] END model__alpha=20, model__fit_intercept=False, model__normalize=True;, score=-6.819 total time=   0.0s
[CV 3/3] END model__alpha=20, model__fit_intercept=False, model__normalize=True;, score=-0.544 total time=   0.0s
[CV 1/3] END model__alpha=20, model__fit_intercept=False, model__normalize=False;, score=-0.553 total time=   0.0s
[CV 2/3] END model__alpha=20, model__fit_intercept=False, model__normalize=False;, score=-6.819 total time=   0.0s
[CV 3/3] END model__alpha=20, model__fit_intercept=False, model__normalize=False;, score=-0.544 total time=   0.0s
[CV 1/3] END model__alpha=30, model__fit_intercept=True, model__normalize=True;, score=-0.558 total time=   0.0s
[CV 2/3] END model__alpha=30, model__fit_intercept=True, model__normalize=True;, score=0.632 total time=   0.0s
[CV 3/3] END model__alpha=30, model__fit_intercept=True, model__normalize=True;, score=-

/Users/mohamednoordeenalaudeen/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/Users/mohamednoordeenalaudeen/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence t

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('numerical_prprocessing',
                                                                         Pipeline(steps=[('imputation',
                                                                                          SimpleImputer()),
                                                                                         ('scale',
                                                                                          MinMaxScaler())]),
                                                                         ['km_driven']),
                                                                        ('categoral_preprocessing',
                                                                         Pipeline(steps=[('pre_preprocess',
                                                                                          CustomTransformer()),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse=False))]),
                                                                         ['name',
                                                                          'fuel_type'])])),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20,
                                          30],
                         'model__fit_intercept': [True, False],
                         'model__normalize': [True, False]},
             scoring='r2', verbose=4)

In [102]:
grid.best_estimator_

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('numerical_prprocessing',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['km_driven']),
                                                 ('categoral_preprocessing',
                                                  Pipeline(steps=[('pre_preprocess',
                                                                   CustomTransformer()),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['name', 'fuel_type'])])),
                ('model', Lasso(alpha=30, normalize=False))])

In [103]:
grid.best_score_

-0.2609349117200723